In [69]:
import pandas as pd
import numpy as np
import gpflow
import matplotlib.pyplot as plt
import utils as u

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [70]:
data = pd.read_csv('../data/gp_weekly_grid_parkslope.csv')
data['DATETIME'] = pd.to_datetime(data['DATETIME'])

data_dict = u.preprocess(data, start_date= '2017/01/01', training_end_date= 26, \
                                                       testing_end_date=12)


Training ends on: 2017-07-02 00:00:00
testing ends on: 2017-09-24 00:00:00


In [67]:
def fit_AR(series, t_pred =12 ):
    ar = AR(endog= series)
    ar_fit = ar.fit(maxlag=1)
    pred = ar.predict(params= ar_fit.params, end=t_pred)
    return pred

def get_MSE(pred,y_test):
    MSE = ((np.square(pred - y_test)).sum()) / len(pred)
    return MSE

def AR_pipe(series, y_test):
    t_pred = len(y_test)
    pred = fit_AR(series.values, t_pred=t_pred)
    return get_MSE(pred, y_test.values)

def run_AR(data_dict):
    ar_df = data_dict['train'].groupby(['DATETIME','GRID_SQUARE'])['COUNT'].sum().unstack()
    ar_df_test = data_dict['test'].groupby(['DATETIME','GRID_SQUARE'])['COUNT'].sum().unstack()
    grid_error = np.array([AR_pipe(ar_df[i],ar_df_test[i]) for i in ar_df])
    return grid_error.sum() 

In [68]:
run_AR(data_dict)

26.627629611257355